In [1]:
from transformers import AutoModelWithLMHead,BertForSequenceClassification, AutoTokenizer, AutoModel,AutoModelForMaskedLM,AutoModelForSequenceClassification
import torch
from torch import nn
import json
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from torch.utils.data import DataLoader,TensorDataset
from transformers import Trainer, TrainingArguments
import pickle
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score,roc_curve
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
model = AutoModelForMaskedLM.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [4]:
model.bert.embeddings.word_embeddings

Embedding(28996, 768, padding_idx=0)

In [5]:
for name, param in model.bert.embeddings.word_embeddings.named_parameters():
    #print(dir(param))
    if param.requires_grad:
        print(name, len(param.data),param.data)

weight 28996 tensor([[-0.0333, -0.0794, -0.0196,  ..., -0.0365, -0.0359,  0.0013],
        [ 0.0125, -0.0182, -0.0349,  ..., -0.0387, -0.0596, -0.0106],
        [-0.0384, -0.0131,  0.0037,  ..., -0.0394, -0.0423, -0.0357],
        ...,
        [-0.0045, -0.0044, -0.0520,  ..., -0.0384, -0.0762, -0.0117],
        [-0.0235,  0.0125, -0.0237,  ..., -0.0818,  0.0034, -0.0393],
        [ 0.0488, -0.0234, -0.0319,  ..., -0.0522, -0.0444, -0.0116]])


In [7]:
for name, param in model.bert.embeddings.word_embeddings.named_parameters():
    #print(dir(param))
    if param.requires_grad:
        print(name, len(param.data),param.data)

weight 29002 tensor([[-0.0333, -0.0794, -0.0196,  ..., -0.0365, -0.0359,  0.0013],
        [ 0.0125, -0.0182, -0.0349,  ..., -0.0387, -0.0596, -0.0106],
        [-0.0384, -0.0131,  0.0037,  ..., -0.0394, -0.0423, -0.0357],
        ...,
        [-0.0015,  0.0085, -0.0136,  ..., -0.0067,  0.0218, -0.0072],
        [-0.0160,  0.0052,  0.0189,  ..., -0.0064, -0.0283,  0.0127],
        [-0.0282,  0.0033,  0.0212,  ..., -0.0097, -0.0260, -0.0052]])


In [5]:
for name, param in model.bert.embeddings.word_embeddings.named_parameters():
    #print(dir(param))
    if param.requires_grad:
        print(name, len(param.data),param.data)

weight 29002 tensor([[-0.0333, -0.0794, -0.0196,  ..., -0.0365, -0.0359,  0.0013],
        [ 0.0125, -0.0182, -0.0349,  ..., -0.0387, -0.0596, -0.0106],
        [-0.0384, -0.0131,  0.0037,  ..., -0.0394, -0.0423, -0.0357],
        ...,
        [-0.0147, -0.0325, -0.0084,  ..., -0.0460,  0.0382,  0.0045],
        [ 0.0046,  0.0099, -0.0027,  ...,  0.0149,  0.0103, -0.0042],
        [ 0.0223,  0.0003,  0.0170,  ...,  0.0172, -0.0408, -0.0061]])


In [6]:
for name, param in model.bert.embeddings.word_embeddings.named_parameters():
    #print(dir(param))
    if param.requires_grad:
        print(name, len(param.data),param.data)

weight 29002 tensor([[-0.0333, -0.0794, -0.0196,  ..., -0.0365, -0.0359,  0.0013],
        [ 0.0125, -0.0182, -0.0349,  ..., -0.0387, -0.0596, -0.0106],
        [-0.0384, -0.0131,  0.0037,  ..., -0.0394, -0.0423, -0.0357],
        ...,
        [-0.0376,  0.0200, -0.0143,  ...,  0.0254,  0.0201, -0.0004],
        [-0.0055, -0.0355,  0.0392,  ..., -0.0206,  0.0027, -0.0159],
        [-0.0111,  0.0308,  0.0389,  ...,  0.0054, -0.0082, -0.0020]])


In [5]:
model.resize_token_embeddings(29002)

Embedding(29002, 768)

In [6]:
model.config.tie_word_embeddings = False

In [5]:
model.config.tie_word_embeddings

True

In [12]:
model.bert.embeddings.word_embeddings.parameters()[-6:] #weight[-6:]

TypeError: 'generator' object is not subscriptable

In [13]:
model.named_parameters()

<generator object Module.named_parameters at 0x0000028B98AEF7B0>

In [2]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
new_tokens = ["interstitial", "fibrosis", "tubular", "atrophy","antibody","T-cell"]
tokenizer.add_tokens(new_tokens)

6

In [2]:
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

In [2]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [2]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [3]:
data = pd.read_csv("data.csv")
# inputs1 = data["train_report"].tolist()
inputs1 = data["train_report_qa"].tolist()
label1 = data["IFTA"].tolist()

In [4]:
label2 = [l for i,l in zip(inputs1,label1) if str(i)!="nan"]
#label = [0 if l in ["nosig","minimal","noinfo"] else (1 if l=="mild" else (2 if l=="moderate" else 3)) for l in label2]
label = []
for l in label2:
    if l == "noinfo":
        label.append(0)
    elif l == "nosig":
        label.append(1)
    elif l == "minimal":
        label.append(2)
    elif l == "mild":
        label.append(3)
    elif l == "moderate":
        label.append(4)
    else:
        label.append(5)
inputs = [i for i in inputs1 if str(i)!="nan"]

In [5]:
train_text, test_text, train_labels, test_labels = train_test_split(
    inputs, label,random_state = 1,stratify=label,test_size=0.2)

In [6]:
class RenalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average="micro")
    precision = precision_score(y_true=labels, y_pred=pred,average="micro")
    f1 = f1_score(y_true=labels, y_pred=pred,average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 

In [7]:
train_encodings = tokenizer(train_text,padding="max_length", truncation=True, 
                            return_tensors="pt",max_length=512)
test_encodings = tokenizer(test_text,padding="max_length", truncation=True, 
                            return_tensors="pt",max_length=512)
train_dataset = RenalDataset(train_encodings, train_labels)
test_dataset = RenalDataset(test_encodings, test_labels)
# model_renal = AutoModelForSequenceClassification.from_pretrained("./mlm_results_largeData/checkpoint-1100",num_labels=6)
# model_renal = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=6)
model_renal = AutoModelForSequenceClassification.from_pretrained("./mlm_results_largeData_extended_tokenizer_lr/checkpoint-1400",num_labels=6)
#model_renal = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1",num_labels=6)


Some weights of the model checkpoint at ./mlm_results_largeData_extended_tokenizer_lr/checkpoint-1400 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mod

In [ ]:
# BERT - ifta cls 
batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_mlm_results_ifta_4labels_largeData_batch16_extoken',          
    num_train_epochs=15,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2580
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.557700,1.388861,0.512409,0.512409,0.512409,0.424480
100,1.231300,1.162278,0.597080,0.597080,0.597080,0.492329
150,1.212000,1.105211,0.613139,0.613139,0.613139,0.502948
200,1.132700,1.053624,0.618978,0.618978,0.618978,0.550526
250,1.072400,0.930673,0.651095,0.651095,0.651095,0.576806
300,0.960700,0.959221,0.665693,0.665693,0.665693,0.594036
350,0.940000,0.925783,0.664234,0.664234,0.664234,0.590649
400,0.936000,0.964335,0.652555,0.652555,0.652555,0.631400
450,0.879100,0.998828,0.637956,0.637956,0.637956,0.564034
500,0.956500,0.960702,0.626277,0.626277,0.626277,0.543292


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-600] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-100
Confi

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-350] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-600
Co

In [ ]:
# KidneyBERT - ifta cls 
batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_mlm_results_ifta_4labels_largeData_batch16_extoken',          
    num_train_epochs=15,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2580
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.462600,1.183166,0.581022,0.581022,0.581022,0.477052
100,1.101800,0.980255,0.633577,0.633577,0.633577,0.562757
150,1.085300,0.928196,0.658394,0.658394,0.658394,0.576777
200,0.965800,0.902720,0.664234,0.664234,0.664234,0.628368
250,0.958000,0.866512,0.678832,0.678832,0.678832,0.648810
300,0.915600,0.905765,0.667153,0.667153,0.667153,0.626471
350,0.837300,0.855402,0.699270,0.699270,0.699270,0.658419
400,0.803300,0.824018,0.718248,0.718248,0.718248,0.696512
450,0.780700,0.822974,0.708029,0.708029,0.708029,0.673226
500,0.806000,0.805589,0.689051,0.689051,0.689051,0.651244


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-100
Confi

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-450] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-600
Co

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-11

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-16

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [8]:
# KidneyBERT - qa ifta cls 
batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_mlm_results_ifta_4labels_largeData_batch16_extoken',          
    num_train_epochs=15,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2580
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.456200,1.151662,0.598540,0.598540,0.598540,0.491674
100,0.795100,0.438924,0.849635,0.849635,0.849635,0.823548
150,0.459400,0.408765,0.891971,0.891971,0.891971,0.878131
200,0.340700,0.387071,0.894891,0.894891,0.894891,0.873957
250,0.331900,0.365728,0.905109,0.905109,0.905109,0.903574
300,0.344200,0.397677,0.886131,0.886131,0.886131,0.868524
350,0.289400,0.352807,0.897810,0.897810,0.897810,0.890348
400,0.297400,0.419329,0.874453,0.874453,0.874453,0.876963
450,0.240900,0.387285,0.899270,0.899270,0.899270,0.891748
500,0.265300,0.388091,0.897810,0.897810,0.897810,0.897468


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-650] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-100
Confi

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-600
Co

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-11

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-16

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-21

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\

In [8]:
# exKidneyBERT - ifta cls 
batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_mlm_results_ifta_4labels_largeData_batch16_extoken',          
    num_train_epochs=15,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2580
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.462900,1.143271,0.598540,0.598540,0.598540,0.493175
100,1.080000,1.083766,0.616058,0.616058,0.616058,0.541205
150,1.075200,0.950542,0.659854,0.659854,0.659854,0.595676
200,0.952100,0.937897,0.630657,0.630657,0.630657,0.579477
250,0.954400,0.900083,0.667153,0.667153,0.667153,0.637690
300,0.915300,0.862881,0.693431,0.693431,0.693431,0.669477
350,0.818500,0.843714,0.691971,0.691971,0.691971,0.635012
400,0.792400,0.832210,0.684672,0.684672,0.684672,0.675337
450,0.765600,0.835042,0.700730,0.700730,0.700730,0.666819
500,0.805000,0.816369,0.671533,0.671533,0.671533,0.632238


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-300] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-100
Confi

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-450] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-600
Co

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-11

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-16

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-21

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\

In [8]:
# exKidneyBERT_lr - qa ifta cls 
batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_mlm_results_ifta_4labels_largeData_batch16_extoken',          
    num_train_epochs=15,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2580
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.451600,1.075703,0.626277,0.626277,0.626277,0.512141
100,0.647800,0.414251,0.891971,0.891971,0.891971,0.880026
150,0.460300,0.437068,0.872993,0.872993,0.872993,0.851261
200,0.400400,0.424771,0.887591,0.887591,0.887591,0.861179
250,0.330000,0.402230,0.890511,0.890511,0.890511,0.876411
300,0.346500,0.361697,0.900730,0.900730,0.900730,0.886696
350,0.330900,0.407881,0.900730,0.900730,0.900730,0.891341
400,0.281100,0.424414,0.881752,0.881752,0.881752,0.886123
450,0.259900,0.417459,0.893431,0.893431,0.893431,0.881994
500,0.236100,0.390827,0.902190,0.902190,0.902190,0.902108


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-800] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-100
Confi

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-600
Co

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-11

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-16

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2050\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-21

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550
Configuration saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550\config.json
Model weights saved in ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2550\pytorch_model.bin
Deleting older checkpoint [fine_mlm_results_ifta_4labels_largeData_batch16_extoken\checkpoint-2500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./fine_mlm_results_ifta_4labels_largeData_batch16_extoken\

In [8]:
# BioBERT - ifta cls 
batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_bioBERT_results_ifta_4labels_largeData_batch16',          
    num_train_epochs=20,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3440
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.489700,1.287165,0.518248,0.518248,0.518248,0.422044
100,1.158700,1.048901,0.610219,0.610219,0.610219,0.500856
150,1.078100,0.960798,0.671533,0.671533,0.671533,0.591547
200,0.978900,0.916042,0.662774,0.662774,0.662774,0.623582
250,0.965100,0.888289,0.699270,0.699270,0.699270,0.649876
300,0.893400,0.889392,0.691971,0.691971,0.691971,0.643470
350,0.823500,0.842183,0.696350,0.696350,0.696350,0.651656
400,0.834300,0.846844,0.702190,0.702190,0.702190,0.679359
450,0.802100,0.880122,0.675912,0.675912,0.675912,0.629390
500,0.832600,0.841846,0.664234,0.664234,0.664234,0.636635


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-50
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-300] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-100
Configuration saved in ./

Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-450] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-600
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-600\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpo

Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_ba

Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1550] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy cons

<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2050] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(T

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2600
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2600\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2600\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2550] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64


KeyboardInterrupt: 

In [ ]:
# BioBERT - qa ifta cls 
batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_bioBERT_results_ifta_4labels_largeData_batch16',          
    num_train_epochs=20,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3440
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.501500,1.382176,0.467153,0.467153,0.467153,0.357851
100,1.051000,0.842015,0.715328,0.715328,0.715328,0.636124
150,0.853300,0.736691,0.748905,0.748905,0.748905,0.665873
200,0.725100,0.679440,0.772263,0.772263,0.772263,0.729630
250,0.552100,0.484167,0.871533,0.871533,0.871533,0.854789
300,0.466900,0.446796,0.859854,0.859854,0.859854,0.846423
350,0.437000,0.521461,0.859854,0.859854,0.859854,0.843272
400,0.389900,0.546551,0.826277,0.826277,0.826277,0.837394
450,0.390600,0.467222,0.865693,0.865693,0.865693,0.846100
500,0.374100,0.392983,0.891971,0.891971,0.891971,0.877525


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-50
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-750] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-100
Configuration saved in ./

Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-450] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-600
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-600\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpo

Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_ba

Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-1550] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy cons

<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2050] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(T

***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2600
Configuration saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2600\config.json
Model weights saved in ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2600\pytorch_model.bin
Deleting older checkpoint [fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2550] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_bioBERT_results_ifta_4labels_largeData_batch16\checkpoint-2650
Configuration sav

In [ ]:
# ClinicalBERT - ifta cls
batch_size = 16
model_cli = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=6)
training_args = TrainingArguments(
    output_dir='./fine_cliBERT_results_ifta_4labels_largeData_batch16',          
    num_train_epochs=15,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_cli,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.554000,1.242224,0.576642,0.576642,0.576642,0.471908
100,1.124300,1.165618,0.613139,0.613139,0.613139,0.501815
150,1.152900,0.963942,0.643796,0.643796,0.643796,0.553779
200,1.032000,0.929965,0.662774,0.662774,0.662774,0.593074
250,0.994200,0.885117,0.665693,0.665693,0.665693,0.596577
300,0.914700,0.883437,0.665693,0.665693,0.665693,0.591820
350,0.864500,0.858240,0.683212,0.683212,0.683212,0.624408
400,0.832700,0.826329,0.694891,0.694891,0.694891,0.685623
450,0.792800,0.875262,0.687591,0.687591,0.687591,0.650343
500,0.870400,0.802420,0.699270,0.699270,0.699270,0.669661


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-50
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-800] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-100
Configuration saved in ./

Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-450] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-600
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-600\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpo

Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-900] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_bat

Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1550] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy cons

<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2050] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(T

In [ ]:
# ClinicalBERT - qa ifta cls
batch_size = 16
model_cli = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=6)
training_args = TrainingArguments(
    output_dir='./fine_cliBERT_results_ifta_4labels_largeData_batch16',          
    num_train_epochs=15,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*4,   
    warmup_steps=50,                
    weight_decay=0.01,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_cli,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,1.492300,1.285793,0.534307,0.534307,0.534307,0.429406
100,1.099200,0.953476,0.668613,0.668613,0.668613,0.594669
150,0.961500,0.826294,0.738686,0.738686,0.738686,0.657862
200,0.683000,0.496641,0.846715,0.846715,0.846715,0.805063
250,0.441200,0.426770,0.883212,0.883212,0.883212,0.865124
300,0.413900,0.423094,0.881752,0.881752,0.881752,0.861376
350,0.373200,0.447873,0.883212,0.883212,0.883212,0.878666
400,0.384700,0.447438,0.865693,0.865693,0.865693,0.867447
450,0.309600,0.414070,0.886131,0.886131,0.886131,0.873508
500,0.352900,0.355984,0.899270,0.899270,0.899270,0.898160


***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-50
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1050] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-100
Configuration saved in .

Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-550\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-450] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-600
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-600\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpo

Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1050\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1000] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1100\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_ba

Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1500] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1600\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-1550] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy cons

<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 64
Saving model checkpoint to ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100
Configuration saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\config.json
Model weights saved in ./fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2100\pytorch_model.bin
Deleting older checkpoint [fine_cliBERT_results_ifta_4labels_largeData_batch16\checkpoint-2050] due to args.save_total_limit
<ipython-input-6-42ad60b603ba>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(T

In [1]:
from transformers import AutoModelWithLMHead,BertForSequenceClassification, AutoTokenizer, AutoModel,AutoModelForMaskedLM,AutoModelForSequenceClassification
import torch
from torch import nn
import json
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from torch.utils.data import DataLoader,TensorDataset
from transformers import Trainer, TrainingArguments
import pickle
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score,roc_curve
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
new_tokens = ["interstitial", "fibrosis", "tubular", "atrophy","antibody","T-cell"]
tokenizer.add_tokens(new_tokens)

6

In [2]:
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

In [2]:
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [3]:
data = pd.read_csv("data.csv")
inputs1 = data["train_report_qa"].tolist()
label1 = data["isRejection"].tolist()

label = [l for i,l in zip(inputs1,label1) if str(i)!="nan"]
inputs = [i for i in inputs1 if str(i)!="nan"]

In [4]:
train_text, test_text, train_labels, test_labels = train_test_split(
    inputs, label,random_state = 1,stratify=label,test_size=0.2)

In [5]:
class RenalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 

In [6]:
train_encodings = tokenizer(train_text,padding="max_length", truncation=True, 
                            return_tensors="pt",max_length=512)
test_encodings = tokenizer(test_text,padding="max_length", truncation=True, 
                            return_tensors="pt",max_length=512)
train_dataset = RenalDataset(train_encodings, train_labels)
test_dataset = RenalDataset(test_encodings, test_labels)
# model_renal = AutoModelForSequenceClassification.from_pretrained("./mlm_results_largeData/checkpoint-1100")
# model_renal = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")
# model_renal = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
# model_renal = AutoModelForSequenceClassification.from_pretrained("./mlm_results_largeData_extended_tokenizer/checkpoint-1100")
model_renal = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [ ]:
# BioBERT - qa cls

batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_exkidBERTcased_results_largeData_batch16_wd1e-5',          
    num_train_epochs=20,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*2,   
    warmup_steps=50,                
    weight_decay=1e-5,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3440
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.348700,0.224976,0.944526,0.000000,0.000000,0.000000
100,0.222600,0.243678,0.944526,0.000000,0.000000,0.000000
150,0.226800,0.210406,0.944526,0.000000,0.000000,0.000000
200,0.189300,0.139218,0.944526,0.000000,0.000000,0.000000
250,0.164200,0.123087,0.964964,0.733333,0.578947,0.647059
300,0.165700,0.245881,0.944526,0.000000,0.000000,0.000000
350,0.152100,0.174738,0.960584,1.000000,0.289474,0.448980
400,0.187600,0.144836,0.959124,1.000000,0.263158,0.416667
450,0.127100,0.129198,0.967883,1.000000,0.421053,0.592593
500,0.161400,0.162830,0.956204,1.000000,0.210526,0.347826


***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
C:\Users\ytc19\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1150] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather tha

Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-500
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-500\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-500\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_

Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-900] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1000
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1000\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1000\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy c

Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1500
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1500\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1500\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy

In [ ]:
# Clinical BERT - qa cls

batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_exkidBERTcased_results_largeData_batch16_wd1e-5',          
    num_train_epochs=20,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*2,   
    warmup_steps=50,                
    weight_decay=1e-5,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3440
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.317800,0.229680,0.944526,0.000000,0.000000,0.000000
100,0.214800,0.263019,0.944526,0.000000,0.000000,0.000000
150,0.232400,0.214161,0.944526,0.000000,0.000000,0.000000
200,0.194400,0.185097,0.944526,0.000000,0.000000,0.000000
250,0.191500,0.150284,0.944526,0.000000,0.000000,0.000000
300,0.192000,0.217008,0.944526,0.000000,0.000000,0.000000
350,0.159700,0.150063,0.957664,0.800000,0.315789,0.452830
400,0.166000,0.132223,0.947445,0.535714,0.394737,0.454545
450,0.134800,0.145540,0.954745,0.652174,0.394737,0.491803
500,0.178400,0.152015,0.957664,1.000000,0.236842,0.382979


***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
C:\Users\ytc19\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-650] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than

***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-350] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-500
Configuration sav

  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-900] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1000
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch

  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1500
Configuration saved in ./fine_exkidBERTcased_results_largeData_b

In [ ]:
# Kidney BERT - qa cls

batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_exkidBERTcased_results_largeData_batch16_wd1e-5',          
    num_train_epochs=20,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*2,   
    warmup_steps=50,                
    weight_decay=1e-5,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3440
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.318500,0.219373,0.944526,0.000000,0.000000,0.000000
100,0.224300,0.265068,0.944526,0.000000,0.000000,0.000000
150,0.247200,0.220932,0.944526,0.000000,0.000000,0.000000
200,0.197300,0.203422,0.944526,0.000000,0.000000,0.000000
250,0.201100,0.166120,0.944526,0.000000,0.000000,0.000000
300,0.187900,0.301379,0.944526,0.000000,0.000000,0.000000
350,0.167400,0.190871,0.953285,0.800000,0.210526,0.333333
400,0.169900,0.134971,0.959124,0.857143,0.315789,0.461538
450,0.141100,0.224407,0.944526,0.000000,0.000000,0.000000
500,0.175000,0.168174,0.944526,0.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
C:\Users\ytc19\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2200] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather tha

***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
C:\Users\ytc19\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-350] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather t

Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-900\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-850] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_

Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1350] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy

Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1850] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1950
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1950\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1950\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1900] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy

Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2350] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2450
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2450\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy

Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2850] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2950
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2950\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2950\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2900] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy

Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3350] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [7]:
# exKidney BERT - qa cls

batch_size = 16
training_args = TrainingArguments(
    output_dir='./fine_exkidBERTcased_results_largeData_batch16_wd1e-5',          
    num_train_epochs=20,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*2,   
    warmup_steps=50,                
    weight_decay=1e-5,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3440
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
50,0.297300,0.224674,0.944526,0.000000,0.000000,0.000000
100,0.218300,0.278130,0.944526,0.000000,0.000000,0.000000
150,0.221800,0.184699,0.944526,0.000000,0.000000,0.000000
200,0.201900,0.163006,0.944526,0.000000,0.000000,0.000000
250,0.147200,0.127365,0.964964,0.705882,0.631579,0.666667
300,0.168400,0.247684,0.944526,0.000000,0.000000,0.000000
350,0.120800,0.193167,0.953285,1.000000,0.157895,0.272727
400,0.157600,0.116394,0.963504,0.782609,0.473684,0.590164
450,0.115300,0.167927,0.962044,0.928571,0.342105,0.500000
500,0.167400,0.127723,0.962044,1.000000,0.315789,0.480000


***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
C:\Users\ytc19\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-50\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1300] due to args.save_total_limit
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-5100] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a te

Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-350] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-500
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-500\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\c

Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-950\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-900] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1000
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1000\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1000\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch

Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1500
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1500\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1500\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_bat

Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1950\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1900] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2000
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2000\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2000\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_bat

Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2450\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2500
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2500\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2500\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_bat

Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2950\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2900] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 32
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3000
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3000\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3000\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_bat

In [7]:
# exKidney BERT - qa cls

batch_size = 8
training_args = TrainingArguments(
    output_dir='./fine_exkidBERTcased_results_largeData_batch16_wd1e-5',          
    num_train_epochs=20,              
    per_device_train_batch_size=batch_size,  
    per_device_eval_batch_size=batch_size*2,   
    warmup_steps=50,                
    weight_decay=1e-5,                          
    logging_steps=50*16/batch_size,
    evaluation_strategy="steps",
    eval_steps=50*16/batch_size,
    load_best_model_at_end=True,
    save_steps = 50*16/batch_size,
    save_total_limit = 1,
    seed = 0
)


trainer = Trainer(
    model=model_renal,                         
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

try:
    
    trainer.train()
except Exception as e:
    print(e)

C:\Users\ytc19\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2738
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6860
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,0.270800,0.276292,0.944526,0.000000,0.000000,0.000000
200,0.239600,0.293186,0.944526,0.000000,0.000000,0.000000
300,0.230500,0.232601,0.944526,0.000000,0.000000,0.000000
400,0.227000,0.205782,0.944526,0.000000,0.000000,0.000000
500,0.235200,0.243196,0.944526,0.000000,0.000000,0.000000
600,0.232400,0.255087,0.944526,0.000000,0.000000,0.000000
700,0.198700,0.281646,0.944526,0.000000,0.000000,0.000000
800,0.228000,0.168213,0.944526,0.000000,0.000000,0.000000
900,0.174800,0.177754,0.953285,1.000000,0.157895,0.272727
1000,0.233000,0.164829,0.944526,0.000000,0.000000,0.000000


***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
C:\Users\ytc19\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-100
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-100\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-100\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather t

Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-800\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-800\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-400] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-900
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-900\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\c

***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1800
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1800\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1800\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1700] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-1900
Configuratio

***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2800
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2800\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2800\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2700] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2900
Configuratio

***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3800
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3800\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3800\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3700] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-3900
Configuratio

***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-4800
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-4800\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-4800\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-4700] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-4900
Configuratio

***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-5800
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-5800\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-5800\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-5700] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-5900
Configuratio

***** Running Evaluation *****
  Num examples = 685
  Batch size = 16
Saving model checkpoint to ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-6800
Configuration saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-6800\config.json
Model weights saved in ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-6800\pytorch_model.bin
Deleting older checkpoint [fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-6700] due to args.save_total_limit
<ipython-input-5-f26e59613042>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./fine_exkidBERTcased_results_largeData_batch16_wd1e-5\checkpoint-2200